### PMIP4 analysis and producing figures for EOS 440 final project 

#### Intake data and filter activity, experiment, variable

In [1]:
# imports should run with given env

import intake
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
# import esm data from server

import intake_esm
url ="https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"

In [3]:
# intake data as xarray

cat = intake.open_esm_datastore(url)

Activity ID for PMIP4 -- experiments used. Lig127k and MidHolocene are taken as the two interglacial datasets, comapred to eachother and the present day Past1000 data.

In [4]:
# filter to PMIP and show experiments 

cat_pmip = cat.search(activity_id = 'PMIP')
cat_pmip.df['experiment_id'].unique()

array(['midHolocene', 'lig127k', 'lgm', 'past1000'], dtype=object)

In [5]:
# seperate each experiemnt out

cat_holo = cat_pmip.search(experiment_id = 'midHolocene', variable_id = 'pr')
cat_lig = cat_pmip.search(experiment_id = 'lig127k', variable_id = 'pr')
cat_now = cat_pmip.search(experiment_id = 'past1000', variable_id = 'pr')

In [6]:
# check the ensemble member numbers for each dataset in order to understand limitations of comparison

print(cat_holo.df['member_id'].unique())
print(cat_lig.df['member_id'].unique())
print(cat_now.df['member_id'].unique())

['r1i1p1f2' 'r1i1p1f3' 'r1i1p1f1']
['r1i1p1f1']
['r1i1p1f1' 'r1i1p1f2']


*Based on the above, we have limited ensemble members for comparison, and need to keep this in mind when discussing results*

#### Convert to dictionary dataset to begin plotting

In [7]:
ds_holo = cat_holo.to_dataset_dict(
    xarray_open_kwargs={"consolidated": True, "decode_times": True, "use_cftime": True}
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [8]:
ds_lig = cat_lig.to_dataset_dict(
    xarray_open_kwargs={"consolidated": True, "decode_times": True, "use_cftime": True}
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [9]:
ds_now = cat_now.to_dataset_dict(
    xarray_open_kwargs={"consolidated": True, "decode_times": True, "use_cftime": True}
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [10]:
# option to un-comment (one at a time) and inspect parameters:

ds_holo
#ds_lig
#ds_now

{'PMIP.IPSL.IPSL-CM6A-LR.midHolocene.Amon.gr': <xarray.Dataset>
 Dimensions:         (lat: 143, lon: 144, time: 6600, dcpp_init_year: 1,
                      member_id: 3, axis_nbounds: 2)
 Coordinates:
   * lat             (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 87.46 88.73 90.0
   * lon             (lon) float32 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
   * time            (time) object 1850-01-16 12:00:00 ... 2399-12-16 12:00:00
   * dcpp_init_year  (dcpp_init_year) float64 nan
   * member_id       (member_id) object 'r1i1p1f1' 'r1i1p1f2' 'r1i1p1f3'
     time_bounds     (time, axis_nbounds) object dask.array<chunksize=(2400, 2), meta=np.ndarray>
 Dimensions without coordinates: axis_nbounds
 Data variables:
     pr              (member_id, dcpp_init_year, time, lat, lon) float32 dask.array<chunksize=(1, 1, 600, 143, 144), meta=np.ndarray>
 Attributes: (12/54)
     CMIP6_CV_version:                 cv=6.2.3.5-2-g63b123e
     Conventions:                      CF-1.7 CMIP-6.2
 